In [1]:
import re
import pandas as pd
from string import ascii_letters as eng_letters
from string import digits
remove_digits = str.maketrans('', '', digits)
from farasa.stemmer import FarasaStemmer
from farasa.segmenter import FarasaSegmenter
import pprint
import time

In [2]:
import sys 
import stop_words_handler
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

In [3]:
yaqutz_pd = pd.read_excel('data/structured/FullYaqut.xlsx', sheet_name='ALL')

In [4]:

yaqutz_pd_types = pd.read_excel('data/structured/FullYaqut.xlsx', sheet_name='Type')
yaqutz_pd_types = set(yaqutz_pd_types['Type_Arabic'])
stemmer = FarasaStemmer(interactive=True)
segmentor = FarasaSegmenter(interactive=True)
places = yaqutz_pd['PlaceName'].tolist()
s = stop_words_handler.Stopwords_Handler()


[2020-11-08 10:07:48,872 - DEBUG]: perform system check...
[2020-11-08 10:07:48,874 - DEBUG]: check java version...
[2020-11-08 10:07:49,113 - DEBUG]: Your java version is 1.8 which is compatiple with Farasa 
[2020-11-08 10:07:49,114 - DEBUG]: check toolkit binaries...
[2020-11-08 10:07:49,115 - INFO]: Dependencies seem to be satisfied..
[2020-11-08 10:07:49,116 - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
[2020-11-08 10:07:49,116 - INFO]: initializing [STEM] task in INTERACTIVE mode...
[2020-11-08 10:07:51,849 - INFO]: task [STEM] is initialized interactively.
[2020-11-08 10:07:51,850 - DEBUG]: perform system check...
[2020-11-08 10:07:51,850 - DEBUG]: check java version...
[2020-11-08 10:07:51,907 - DEBUG]: Your java version is 1.8 which is compatiple with Farasa 
[2020-11-08 10:07:51,908 - DEBUG]: check toolkit binaries...
[2020-11-08 10:07:51,909 - INFO]: Dependencies seem to be satisfied..
[2020-11

In [10]:
def search_place(palce):
    print("\n".join(s for s in places if palce in s))


In [11]:
# from farasa.ner import FarasaNamedEntityRecognizer 
# ner = FarasaNamedEntityRecognizer(interactive=True)

In [8]:
place_names = yaqutz_pd[yaqutz_pd['PlaceName'].apply(lambda x: len(x.split(' ')) < 3)]['PlaceName']
place_names = place_names.apply(lambda x:stemmer.stem(x) )
place_names = set(place_names)

AttributeError: 'float' object has no attribute 'split'

In [32]:
df = yaqutz_pd['PlaceName'].dropna()
df = df.apply(lambda x:clean_text(x))
place_names = df[df.apply(lambda x: len(x.split(' ')) < 3)]
place_names = set(place_names)

In [33]:
place_names

{'منابض',
 'شاوخران',
 'سكلكند',
 'فيف',
 'حمير',
 'بذى',
 'ملزق',
 'خاوران',
 'ثقب',
 'دمعان',
 'دارة رفرف',
 'عملة',
 'فراش',
 'حمقتان',
 'أيدم',
 'هط',
 'مظلم',
 'برزاباذان',
 'سيارى',
 'جرجانية',
 'ثأي',
 'ربض حرب',
 'خيال',
 'تاشكوط',
 'دير متى',
 'روضة كري',
 'فج زيدان',
 'أشبورة',
 'نخذ',
 'شوحطان',
 'جالص',
 'جاس',
 'أسيل',
 'أبسوج',
 'خدفران',
 'جذمان',
 'أدي',
 'عرو',
 'قصر زيت',
 'هورقان',
 'أبين',
 'دمقش',
 'قمار',
 'دارة خوج',
 'زهيوط',
 'نازي',
 'فراخ',
 'رقب',
 'بازبدى',
 'حوراء',
 'صان',
 'بلنسي',
 'ميكع',
 'أبرق بادي',
 'صر',
 'رهو',
 'ضربيط',
 'أشا',
 'زخم',
 'جلي',
 'مراهط',
 'كاوردان',
 'رميش',
 'مضارج',
 'شمطتان',
 'مربولة',
 'داجي',
 'طاق حجام',
 'كول',
 'لحوظ',
 'تبني',
 'دير بشر',
 'موهبة',
 'أندق',
 'راميشن',
 'شلم',
 'إدوي',
 'زرائب',
 'سرنة',
 'ظليف',
 'حر بر',
 'أللان',
 'نطح',
 'صيمور',
 'أرال',
 'كرزبان',
 'أساود',
 'صالف',
 'أخرم',
 'طنبذة',
 'جوفر',
 'زهدم',
 'إسكندرون',
 'ريوند',
 'خرم',
 'أزمور',
 'وير',
 'كندير',
 'محرقة',
 'قصر رمان',
 'جاجرم',
 'أرو

In [29]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    Ifoo [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None

    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)
        
        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)
        
        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [30]:
def clean_text(sentence, hard=False,stem=True):
    words = []

    if not hard:
        punc = '''</>~%#`÷×؛_()*&^%][ـ/"؟'{}~¦+|!”…“–ـ'''
        punc = punc + eng_letters
    else:
        english_punctuations = string.punctuation
        arabic_punctuations = '''`÷×؛_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
        punc = english_punctuations + arabic_punctuations

    for w in sentence.split(' '):

        # Cleaning Numbers
        w = w.translate(remove_digits)

        # Removing the punctuations
        for ch in punc:
            w = w.replace(ch, '')

        if any(sep in w for sep in ['.', '،', ':']):
            w = w[0:-1] + ' ' + w[-1]

        if not re.search(r'[a-zA-Z]', w):
            ## here we should handle ب and بال 
            if stem:
                words.append(stemmer.stem(w))
            else:
                words.append(w)

    return ' '.join(words)


###### For each word in each sentence , check if a word is a Place Name, Type Of Place, or a Preposition word. with respect to Bi-grams(for now )

In [163]:
def _TPP_recognition(text,stem=True):

    text = clean_text(text,stem)
    entities = []
    entity = {}
    pattern = re.compile(r" ، | : | . ")
    for sentence in pattern.split(text):
        entity = {}
        
        #uni-grams
        for index , word in zip(range(len(sentence.split(' '))),sentence.split(' ')):
            
            
            token = stemmer.stem(word)
            next_token=''
            if index < len(sentence.split(' '))-1 :
                next_token = (sentence.split(' ')[index+1])

            if (type_exists(token)):
                # TODO:
                if 'type' in entity:
                    entity['type'].append(token)
                else:
                    entity['type'] = [token] 
            elif token in place_names:
                    
                # TODO:
                if token+ ' ' + next_token not in place_names:
                    if 'name' in  entity :

                        entity['name'].append(token)
                    else:
                        entity['name'] = [token]



                    
            if s.stop_word_type(token) == 0:
                # TODO:
                entity['s_prep'] = token
        #bi-grams
        n_grams = get_ngrams(sentence,2)
        for token in n_grams:
            if token in place_names:
                if 'name' in  entity :
                    entity['name'].append(token)
                else:
                    entity['name'] = [token]
            
            
        if entity:
            
            entities.append(entity)

    return entities

In [58]:
def type_exists(type):
    return type in yaqutz_pd_types

In [80]:
def get_ngrams(text, n ):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

In [59]:
_TPP_recognition(clean_example1)

NameError: name 'clean_example1' is not defined

In [60]:
text = 'بالجيم المكسورة والنون الساكنة وقاف وألف ونون : وهي قرية من قرى سرخس، ينسب إليها أبو الفضل محمد بن عبد الواحد الآجنقاني ، والعجم آجنكان . '

example1 = '''# قرية من نواحي بانياس من أعمال دمشق بين دمشق والساحل. وآبل أيضا، آبل
<br />~~السوق: قرية كبيرة في غوطة دمشق، من ناحية الوادي، ينسب إليها أبو طاهر
<br />~~الحسين بن محمد بن الحسين بن عامر بن احمد يعرف بابن خراشة الأنصاري
<br />~~الخزرجي المقري الآبلي، إمام جامع دمشق، قرأ القرآن على أبي المظفر الفتح
<br />~~بن برهان الأصبهاني وأقرانه، وروى عن أبي علي الحسين بن ابراهيم بن جابر،
<br />~~يعرف بابن أبي الزمزم الفرائضي، وأبي بكر عبد الله بن محمد بن عبد الله بن
<br />~~هلال الحنائي، واحمد بن محمد المؤذن أبي القاسم، وأبي بكر الميانجي، وأبي
<br />~~عبد الله محمد بن عبد الله بن ذكوان، وأبي همام محمد بن ابراهيم بن عبد
<br />~~الله الحافظ، وروى عنه أبو عبد الله بن أبي الحديد، ومحمد ابن أحمد بن أبي
<br />~~الصفر الأنباري، وأبو سعد السمان، وأبو محمد عبد العزيز الكتاني، وقال:
<br />~~توفي شيخنا أبو طاهر الآبلي في سابع عشر ربيع الآخر سنة 428 وكان ثقة نبيلا
<br />~~مأمونا. وقال أحمد بن منير:
<br /># حي الديار على علياء جيرون، % % مهوى الهوى ومغاني الخرد العين
<br /># مراد لهوي، إذ كفي مصرفة % % أعنة العيش في فتح الميادين
<br /># فالنير بين، فمقرى، فالسرير، فخم % % رايا، فجو حواشي جسر جسرين
<br /># فالقصر، فالمرج Milestone300، فالميدان، فالشرف ال % % أعلى، فسطرا، فجرنان، فقلبين
<br /># فالماطرون، فداريا، فجارتها % % فآبل، فمغاني دير قانون
<br /># تلك المنازل، لا وادي الأراك، ولا % % رمل المصلى، ولا أثلاث يبرين
<br /># وآبل أيضا من قرى حمص من جهة القبلة، بينها وبين حمص نحو ميلين.'''

In [61]:
clean_example1 = clean_text(example1)

In [62]:
print(clean_example1)
clean_example1 = ' '.join(clean_example1.split("\n"))
clean_example1.replace('.','.')


 قرية من ناحية بانياس من عمل دمشق بين دمشق ساحل . آبل أيض ، آبل سوق : قرية كبير في غوط دمشق ، من ناحية وادي ، نسب إلى أبو طاهر حسين ابن محمد ابن حسين ابن عامر ابن أحمد عرف ابن خراش أنصاري خزرجي مقر آبلي ، إمام جامع دمشق ، قرأ قرآن على أب مظفر فتح ابن برهان الأصبهاني قرن ، روى عن أب علي حسين ابن إبراهيم ابن جابر ، عرف ابن أب زمزم فرائضي ، أب بكر عبد الله ابن محمد ابن عبد الله ابن هلال حنائي ، أحمد ابن محمد مؤذن أب قاسم ، أب بكر ميانجي ، أب عبد الله محمد ابن عبد الله ابن ذكوان ، أب همام محمد ابن إبراهيم ابن عبد الله حافظ ، روى عن أبو عبد الله ابن أب حديد ، محمد ابن أحمد ابن أب صفر أنباري ، أبو سعد سمان ، أبو محمد عبد عزيز كتاني ، قال : توفى شيخ أبو طاهر آبلي في سابع عشر ربيع آخر سنة  كان ثقة نبيل مأمون . قال أحمد ابن منير :  حي دار على علياء جير ،   مهوى هوى مغاني خرد عين  مراد هو ، إذ كف مصرف   أعن عيش في فتح ميدان  نير بين ، مقرى ، سرير ، فخم   رأي ، فجو حاشية جسر جسر  قصر ، مرج ، ميدان ، شرف ال   أعلى ، سطر ، فجرنان ، فقلبين  ممطر ، داري ، جاري   آبل ، مغاني دير قانون  ذلك منزل ، لا و

' قرية من ناحية بانياس من عمل دمشق بين دمشق ساحل . آبل أيض ، آبل سوق : قرية كبير في غوط دمشق ، من ناحية وادي ، نسب إلى أبو طاهر حسين ابن محمد ابن حسين ابن عامر ابن أحمد عرف ابن خراش أنصاري خزرجي مقر آبلي ، إمام جامع دمشق ، قرأ قرآن على أب مظفر فتح ابن برهان الأصبهاني قرن ، روى عن أب علي حسين ابن إبراهيم ابن جابر ، عرف ابن أب زمزم فرائضي ، أب بكر عبد الله ابن محمد ابن عبد الله ابن هلال حنائي ، أحمد ابن محمد مؤذن أب قاسم ، أب بكر ميانجي ، أب عبد الله محمد ابن عبد الله ابن ذكوان ، أب همام محمد ابن إبراهيم ابن عبد الله حافظ ، روى عن أبو عبد الله ابن أب حديد ، محمد ابن أحمد ابن أب صفر أنباري ، أبو سعد سمان ، أبو محمد عبد عزيز كتاني ، قال : توفى شيخ أبو طاهر آبلي في سابع عشر ربيع آخر سنة  كان ثقة نبيل مأمون . قال أحمد ابن منير :  حي دار على علياء جير ،   مهوى هوى مغاني خرد عين  مراد هو ، إذ كف مصرف   أعن عيش في فتح ميدان  نير بين ، مقرى ، سرير ، فخم   رأي ، فجو حاشية جسر جسر  قصر ، مرج ، ميدان ، شرف ال   أعلى ، سطر ، فجرنان ، فقلبين  ممطر ، داري ، جاري   آبل ، مغاني دير قانون  ذلك منزل ، لا 

In [164]:
df5 = yaqutz_pd.copy() 
yaqutz_pd

,#,PlaceName,Description,Latitude,Longitude,Type1,Type2,TypeFull,PlaceType,TypeEng,...,wikilat,wikilong,TurLat,TurLong,TurNameTr,TurNameEng,TurNameAr,TurType,TurRegionEng,TurRegionTr
0,0,آبار الأعراب,# جمع بئر. يقال في جمعها آبار وبئار وأبار: موض...,NaN,NaN,موضع,موضع,موضع,موضع,place,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,آبج,# بفتح الهمزة وبعد الألف باء موحدة مفتوحة وجيم...,NaN,NaN,موضع,موضع,موضع,موضع,place,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,آبر,# بفتح الهمزة وسكون الألف وضم الباء الموحدة ور...,NaN,NaN,قرية,قرية,قرية,قرية,village,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,آبسكون,# بفتح الهمزة وسكون الألف وفتح الباء الموحدة و...,37.027720,54.043270,بليدة,بليدة,بليدة,بليدة,small town (f),...,NaN,NaN,37.02772,54.04327,Ābaskūn,Abaskun,آبسكون,towns,Daylam,al-Daylam
4,4,آبل,# بفتح الهمزة وبعد الألف باء مكسورة ولام:\n<br...,34.637222,36.676111,أربعة مواضع وفي الحديث,أربعة مواضع وفي الحديث,أربعة مواضع وفي الحديث,أربعة مواضع,NaN,...,34.637222,36.676111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12449,12449,يونان,# بالضم ثم السكون، ونونين بينهما ألف:\n<br />#...,39.000000,22.000000,موضع,موضع,موضع,موضع,place,...,39.000000,22.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12450,12450,يؤيؤ,# بالضم ثم السكون ثم مثله، يوم يؤيؤ: وهو يوم ا...,NaN,NaN,وهو يوم الأواق من,وهو يوم الأواق من,يوم الأواق من,يوم,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12451,12451,يهرع,# بالفتح، قوله تعالى: وجاءه قومه يهرعون إليه، ...,NaN,NaN,وجاءه,وجاءه,وجاءه,وجاءه,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12452,12452,يهودية,# نسبة إلى اليهود في موضعين: أحدهما محلة بجرجا...,35.934700,64.838570,أحدهما,أحدهما,أحدهما,أحدهما,NaN,...,32.500000,34.900000,35.93470,64.83857,"al-Yahūdiyyaŧ, Ǧakūḏān","al-Yahudiyya, Jakudhan",اليهودية,waystations,Khurasan,Ḫurāsān


In [165]:
df5.columns

Index(['#', 'PlaceName', 'Description', 'Latitude', 'Longitude', 'Type1',
       'Type2', 'TypeFull', 'PlaceType', 'TypeEng', 'TrHebrew', 'TrEnglish',
       'wikidataID', 'WikiNameEng', 'WikiNamesAlt', 'ModernCountry',
       'ModernClass', 'elevation', 'geonames_id', 'wikilat', 'wikilong',
       'TurLat', 'TurLong', 'TurNameTr', 'TurNameEng', 'TurNameAr', 'TurType',
       'TurRegionEng', 'TurRegionTr'],
      dtype='object')

In [166]:
tick = time.time()
df5['step1']  = df5['Description'].apply(lambda x : _TPP_recognition(x))
tack = time.time()
        
print(tack-tick)

KeyboardInterrupt: 

In [152]:
values = df5['step1'].values


In [153]:
values

array([list([{'name': ['جمع'], 'type': ['بئر']}, {'name': ['قال', 'في', 'جمع', 'أبار'], 's_prep': 'في', 'type': ['بئر']}, {'type': ['موضع'], 'name': ['بين', 'أجفر', 'يد'], 's_prep': 'بين'}, {'name': ['خمس', 'ميل', 'من', 'أجفر'], 's_prep': 'من'}, {'type': ['بئر']}, {'type': ['كورة', 'كور'], 'name': ['من', 'واسط'], 's_prep': 'من'}]),
       list([{'name': ['بعد']}, {'type': ['موضع', 'بلد'], 'name': ['في', 'عبد', 'ابن', 'ابن'], 's_prep': 'في'}, {'name': ['عن', 'أب']}, {'name': ['أخرج', 'حديث']}, {'name': ['لا', 'آبه', 'زيد']}, {'name': ['قال', 'في', 'رمى'], 's_prep': 'في'}, {'name': ['لا']}, {'name': ['أعلم']}]),
       list([{'name': ['ضم'], 's_prep': 'وراء'}, {'type': ['قرية', 'قرية'], 'name': ['من', 'سجستان'], 's_prep': 'من'}, {'name': ['حسن', 'ابن', 'ابن', 'ابن', 'عاصم']}, {'name': ['شيخ', 'من', 'حديث'], 's_prep': 'من'}, {'name': ['ل', 'نفيس', 'كبير', 'في', 'خبر', 'أب', 'عبد', 'ابن'], 's_prep': 'في'}, {'name': ['عن']}, {'name': ['في', 'كل'], 's_prep': 'في'}, {'name': ['كان', 'مصر', 'ش

## Most Frequent False Pos detected by _TPP Func. 

In [154]:
_list = []
for value in values:
    for item in value:
        if 'name' in item:
            _list.append(item['name'])
from collections import Counter
words = Counter(c for clist in _list for c in clist)
sortred = sorted(words.items(), key=lambda item: (-item[1], item[0]))

In [155]:
sortred

[('من', 34610),
 ('ابن', 34120),
 ('في', 25963),
 ('قال', 24161),
 ('كان', 10145),
 ('عن', 7644),
 ('أن', 7523),
 ('هو', 7172),
 ('بين', 6656),
 ('أب', 6471),
 ('عبد', 5670),
 ('لا', 5433),
 ('ثم', 5367),
 ('قد', 4950),
 ('هذا', 4233),
 ('ذكر', 4035),
 ('هي', 3900),
 ('يوم', 3662),
 ('بنون', 3613),
 ('لم', 3499),
 ('أول', 3392),
 ('آخر', 3376),
 ('سنة', 3247),
 ('اسم', 2839),
 ('أحمد', 2599),
 ('بعد', 2578),
 ('علي', 2414),
 ('كل', 2351),
 ('ضم', 2271),
 ('كسر', 2200),
 ('حسن', 2159),
 ('ملك', 2131),
 ('رأى', 2004),
 ('نون', 2002),
 ('حتى', 1827),
 ('بكر', 1808),
 ('أو', 1665),
 ('سمى', 1600),
 ('رجل', 1546),
 ('دار', 1525),
 ('قرب', 1524),
 ('جمع', 1519),
 ('عمر', 1501),
 ('مثل', 1435),
 ('وجد', 1352),
 ('أحد', 1328),
 ('ليس', 1289),
 ('قبل', 1269),
 ('يمن', 1264),
 ('شعر', 1222),
 ('عمرو', 1218),
 ('مصر', 1211),
 ('واحد', 1198),
 ('ساكن', 1161),
 ('مكة', 1151),
 ('عرف', 1133),
 ('سلم', 1119),
 ('باب', 1106),
 ('خرج', 1105),
 ('بغداد', 1101),
 ('مع', 1093),
 ('أمر', 1060),
 ('سعد', 10

#### Remove most frequent items in prev result, in oroder to get "Cleaner results"

In [157]:
# filter_palcename  = place_names.copy()
tobe_removed=[]
for item in sortred[:70] : 
    assert (item[0] in place_names)
    tobe_removed.append(item[0])


In [161]:
import csv
with open('filter_palcename.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(list(filter_palcename))

In [308]:
tick = time.time()
df5['step1']  = df5['Description'].apply(lambda x : _TPP_recognition(x))
tack = time.time()

print(tack-tick)

520.1183140277863


In [310]:
df5['step1'].to_csv('tpp_output.csv')

c:\users\odin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [118]:
pattern = re.compile(r" ، | : | . ")

In [119]:
pattern.split(clean_example1)

[' قرية من ناحية بانياس من عمل دمشق بين دمشق ساحل',
 'آبل أيض',
 'آبل سوق',
 'قرية كبير في غوط دمشق',
 'من ناحية وادي',
 'نسب إلى أبو طاهر حسين ابن محمد ابن حسين ابن عامر ابن أحمد عرف ابن خراش أنصاري خزرجي مقر آبلي',
 'إمام جامع دمشق',
 'قرأ قرآن على أب مظفر فتح ابن برهان الأصبهاني قرن',
 'روى عن أب علي حسين ابن إبراهيم ابن جابر',
 'عرف ابن أب زمزم فرائضي',
 'أب بكر عبد الله ابن محمد ابن عبد الله ابن هلال حنائي',
 'أحمد ابن محمد مؤذن أب قاسم',
 'أب بكر ميانجي',
 'أب عبد الله محمد ابن عبد الله ابن ذكوان',
 'أب همام محمد ابن إبراهيم ابن عبد الله حافظ',
 'روى عن أبو عبد الله ابن أب حديد',
 'محمد ابن أحمد ابن أب صفر أنباري',
 'أبو سعد سمان',
 'أبو محمد عبد عزيز كتاني',
 'قال',
 'توفى شيخ أبو طاهر آبلي في سابع عشر ربيع آخر سنة  كان ثقة نبيل مأمون',
 'قال أحمد ابن منير',
 ' حي دار على علياء جير',
 '  مهوى هوى مغاني خرد عين  مراد هو',
 'إذ كف مصرف',
 'أعن عيش في فتح ميدان  نير بين',
 'مقرى',
 'سرير',
 'فخم',
 'رأي',
 'فجو حاشية جسر جسر  قصر',
 'مرج',
 'ميدان',
 'شرف ال',
 'أعلى',
 'سطر',
 'فج

In [121]:
clean_example2 = clean_text(example1,stem=False)
clean_example2 = ' '.join(clean_example2.split("\n"))
pattern.split(clean_example2)

[' قرية من نواحي بانياس من أعمال دمشق بين دمشق والساحل',
 'وآبل أيضا',
 'آبل  السوق',
 'قرية كبيرة في غوطة دمشق',
 'من ناحية الوادي',
 'ينسب إليها أبو طاهر  الحسين بن محمد بن الحسين بن عامر بن احمد يعرف بابن خراشة الأنصاري  الخزرجي المقري الآبلي',
 'إمام جامع دمشق',
 'قرأ القرآن على أبي المظفر الفتح  بن برهان الأصبهاني وأقرانه',
 'وروى عن أبي علي الحسين بن ابراهيم بن جابر،',
 'يعرف بابن أبي الزمزم الفرائضي',
 'وأبي بكر عبد الله بن محمد بن عبد الله بن  هلال الحنائي',
 'واحمد بن محمد المؤذن أبي القاسم',
 'وأبي بكر الميانجي',
 'وأبي  عبد الله محمد بن عبد الله بن ذكوان',
 'وأبي همام محمد بن ابراهيم بن عبد  الله الحافظ',
 'وروى عنه أبو عبد الله بن أبي الحديد',
 'ومحمد ابن أحمد بن أبي  الصفر الأنباري',
 'وأبو سعد السمان',
 'وأبو محمد عبد العزيز الكتاني',
 'وقال:',
 'توفي شيخنا أبو طاهر الآبلي في سابع عشر ربيع الآخر سنة  وكان ثقة نبيلا  مأمونا',
 'وقال أحمد بن منير:',
 ' حي الديار على علياء جيرون',
 '  مهوى الهوى ومغاني الخرد العين',
 'مراد لهوي',
 'إذ كفي مصرفة',
 'أعنة العيش في فتح الميادين

In [164]:
result = _TPP_recognition(clean_example1)


In [175]:
def write_excel(filename,sheetname,dataframe):
    with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer: 
        workBook = writer.book
        try:
            workBook.remove(workBook[sheetname])
        except:
            print("Worksheet does not exist")
        finally:
            dataframe.to_excel(writer, sheet_name=sheetname,index=False)
            writer.save()


# Remove redundate "AL-ALtaareef" form PlaceName 

In [176]:

df = yaqutz_pd.copy()                                   
df['PlaceName'] = df['PlaceName'].apply(lambda x : segmentor.desegment('+'.join(segmentor.segment(x).split('+')[1:] if x.startswith('ال') else x )))
# append_df_to_excel('data\structured\FullYaqut.xlsx',df,sheet_name='ALL',truncate_sheet=True)
write_excel('data\structured\FullYaqut.xlsx','ALL',df)





In [295]:
[]
# print('دمشق' in place_names) 15, 394,تل أعرن أبرق البادي
# segmentor.segment('البقال')
# stemmer.stem('البقال')
df = yaqutz_pd.loc[yaqutz_pd['PlaceName'] == 'ثلاث']
df

,#,PlaceName,Description,Latitude,Longitude,Type1,Type2,TypeFull,PlaceType,TypeEng,...,wikilat,wikilong,TurLat,TurLong,TurNameTr,TurNameEng,TurNameAr,TurType,TurRegionEng,TurRegionTr
2752,2752,ثلاث,# بالضم، بلفظ المعدول عن ثلاثة: موضع أراه من د...,NaN,NaN,موضع,موضع,موضع,موضع,place,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
bi_grams_df = bi_grams_df.apply(lambda x: clean_text(x))
bi_grams_df['result'] = bi_grams_df.apply(lambda x: _TPP_recognition(x))

c:\users\odin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
bi_grams_df['result'].to_csv('bi_grams_test.csv')

c:\users\odin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [167]:
'ب' in place_names

True

In [168]:
search_place('ب')

TypeError: argument of type 'float' is not iterable

In [74]:

bi_places= yaqutz_pd[yaqutz_pd['PlaceName'].apply(lambda x: len(x.split(' ')) == 2)]['PlaceName'].apply(lambda x: x.split(' ')[0])
bi_places.value_counts().to_csv('bi_grams_1stWord.csv')


0        آبار الأعراب
1                 آبج
2                 آبر
3              آبسكون
4                 آبل
             ...     
12449           يونان
12450            يؤيؤ
12451            يهرع
12452          يهودية
12453            ييعث
Name: PlaceName, Length: 12448, dtype: object

In [31]:
bi_grams_df = pd.read_csv('data/structured/2- grams.csv')  

In [32]:
bi_grams_df['PlaceName'].to_csv('bi_grams_place_names.csv')

c:\users\odin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [52]:
bi_grams_df[bi_grams_df['PlaceName'].str.startswith('ال')]['PlaceName']

26               الأبرق الفرد
65              الإستان العال
164              البحر المحيط
178           البحيرة المنتنة
356              البيت الحرام
365              البيت العتيق
439            التين والزيتون
441            الثنية البيضاء
460          الجزائر الخالدات
464           الجزيرة الخضراء
484        الحبل Milestone300
485              الحجر الأسود
495             الحرة الرجلاء
516            الحريم الطاهري
775              الدير الخصيب
887              الديرة البيض
905        الران Milestone300
940             الركن اليماني
1092         السلمية والبرشام
1127            الشاه والعروس
1183           العارضة السفلى
1185    العاصمية Milestone300
1220        العم Milestone300
1266             القصر الأبيض
1346          القنطرة الجديدة
1357             الكبش والأسد
1463            المسجد الحرام
1465            المشعر الحرام
1466     المصامة Milestone300
1644             الهني والمري
Name: PlaceName, dtype: object

# Type formatting


In [37]:
pd_types = pd.read_excel('data/structured/FullYaqut.xlsx', sheet_name='Type')

In [41]:
pd_types = set(pd_types['Type_Arabic'].dropna())

In [42]:
pd_types

{'أرض',
 'أطم',
 'أمة',
 'أودية',
 'إقليم',
 'الغوطة',
 'بئر',
 'بحر',
 'بحيرة',
 'بركة',
 'بساتين',
 'بلاد',
 'بلد',
 'بلدان',
 'بلدة',
 'بلدتان',
 'بليد',
 'بليدة',
 'تل',
 'ثغر',
 'ثغور',
 'ثور',
 'جامع',
 'جبال',
 'جبل',
 'جبلة',
 'جزائر',
 'جزيرة',
 'جسر',
 'جند',
 'جيل',
 'حبل',
 'حد',
 'حصن',
 'حصون',
 'خان',
 'خيال',
 'ديار',
 'دير',
 'رابيتان',
 'ربض',
 'رستاق',
 'رمل',
 'رية',
 'زاب',
 'سحل',
 'سقيفة',
 'سوق',
 'شجر',
 'شعب',
 'صقع',
 'صنم',
 'ضيعة',
 'طريق',
 'عمل',
 'عمود',
 'عين',
 'فرضة',
 'قبيلة',
 'قرى',
 'قرية',
 'قريتان',
 'قريتين',
 'قصبة',
 'قصر',
 'قصران',
 'قلاع',
 'قلعة',
 'قلعتين',
 'كرخ',
 'كور',
 'كورة',
 'كورتان',
 'لغة',
 'ما',
 'ماء',
 'ماءة',
 'مازحين',
 'محال',
 'محلة',
 'مدينة',
 'مرج',
 'مرحلة',
 'منارة',
 'منخفض',
 'منزل',
 'مواضع',
 'موضع',
 'موضعان',
 'ناحية',
 'نبع',
 'نهر',
 'نهران',
 'هضبة',
 'واد',
 'وادي',
 'واديان',
 'ولاية'}

In [47]:
bi_gram_first_tokens = pd.read_csv('bi_grams_1stWord.csv')

In [52]:
bgram_set= set(bi_gram_first_tokens.name)

In [53]:
bgram_set & pd_types 

{'أرض',
 'أطم',
 'بئر',
 'بحر',
 'بحيرة',
 'بركة',
 'تل',
 'جبل',
 'جزائر',
 'جزيرة',
 'جسر',
 'حصن',
 'خان',
 'ديار',
 'دير',
 'ربض',
 'سوق',
 'شعب',
 'عين',
 'فرضة',
 'قصر',
 'قلعة',
 'كرخ',
 'كور',
 'ماء',
 'مدينة',
 'مرج',
 'منارة',
 'نهر',
 'وادي'}

In [54]:
bgram_set - pd_types 

{'؟',
 'آبار',
 'آجام',
 'آل',
 'أبارق',
 'أبرق',
 'أبرقا',
 'أبضع',
 'أبو',
 'أجبال',
 'أجمة',
 'أجناد',
 'أحامر',
 'أحجار',
 'أخشن',
 'أذرع',
 'أرطة',
 'أرم',
 'أزاد',
 'أزاذ',
 'أسبار',
 'أستاذ',
 'أسود',
 'أشلاء',
 'أضاءة',
 'أطرابلس',
 'أعلاق',
 'أقناب',
 'أم',
 'أمرة',
 'أندة',
 'أهل',
 'أوارة:',
 'أوانا',
 'إرام',
 'إرم',
 'إستان',
 'إيران',
 'ابن',
 'ابنا',
 'الأبرق',
 'الإستان',
 'البحر',
 'البحيرة',
 'البيت',
 'التين',
 'الثنية',
 'الجزائر',
 'الجزيرة',
 'الحبل',
 'الحجر',
 'الحرة',
 'الحريم',
 'الدير',
 'الديرة',
 'الران',
 'الركن',
 'السلمية',
 'الشاه',
 'العارضة',
 'العاصمية',
 'العم',
 'القصر',
 'القنطرة',
 'الكبش',
 'المسجد',
 'المشعر',
 'المصامة',
 'الهني',
 'با',
 'باب',
 'باجرا:',
 'باذان',
 'باوري',
 'باي',
 'بج',
 'بداكر',
 'براز',
 'براق',
 'برج',
 'برز',
 'برقاء',
 'برقة',
 'برك',
 'بستان',
 'بصاق:',
 'بصير',
 'بض',
 'بطن',
 'بغد',
 'بقيع',
 'بلقاء',
 'بنات',
 'بنتا',
 'بنج',
 'بنو',
 'بهمن',
 'بوزن',
 'بون',
 'بيت',
 'بيجن',
 'بيروز',
 'بيعة',
 'بين',
 'تخاران',


In [84]:
def find_ngram_in_text(text,ngram):
    n_grams = get_ngrams(text,len(ngram.split(' ')))
    return [ngram in n_grams]


In [115]:
def find_mn_patter(text):
    tokens = text.split(' ')
    indxies=[]
    for _type in pd_types:
        for ix,token in zip(range(len(tokens)),tokens):
            if ix < len(text.split(' ')) - 4 :
                if token == _type and tokens[ix+1] == 'من' and tokens[ix+2] == _type :
                    indxies.append(ix)
    return indxies if not None else None

In [81]:
get_ngrams(clean_example1,2)

['قرية من',
 'من ناحية',
 'ناحية بانياس',
 'بانياس من',
 'من عمل',
 'عمل دمشق',
 'دمشق بين',
 'بين دمشق',
 'دمشق ساحل',
 'ساحل .',
 '. آبل',
 'آبل أيض',
 'أيض ،',
 '، آبل',
 'آبل سوق',
 'سوق :',
 ': قرية',
 'قرية كبير',
 'كبير في',
 'في غوط',
 'غوط دمشق',
 'دمشق ،',
 '، من',
 'من ناحية',
 'ناحية وادي',
 'وادي ،',
 '، نسب',
 'نسب إلى',
 'إلى أبو',
 'أبو طاهر',
 'طاهر حسين',
 'حسين ابن',
 'ابن محمد',
 'محمد ابن',
 'ابن حسين',
 'حسين ابن',
 'ابن عامر',
 'عامر ابن',
 'ابن أحمد',
 'أحمد عرف',
 'عرف ابن',
 'ابن خراش',
 'خراش أنصاري',
 'أنصاري خزرجي',
 'خزرجي مقر',
 'مقر آبلي',
 'آبلي ،',
 '، إمام',
 'إمام جامع',
 'جامع دمشق',
 'دمشق ،',
 '، قرأ',
 'قرأ قرآن',
 'قرآن على',
 'على أب',
 'أب مظفر',
 'مظفر فتح',
 'فتح ابن',
 'ابن برهان',
 'برهان الأصبهاني',
 'الأصبهاني قرن',
 'قرن ،',
 '، روى',
 'روى عن',
 'عن أب',
 'أب علي',
 'علي حسين',
 'حسين ابن',
 'ابن إبراهيم',
 'إبراهيم ابن',
 'ابن جابر',
 'جابر ،',
 '، عرف',
 'عرف ابن',
 'ابن أب',
 'أب زمزم',
 'زمزم فرائضي',
 'فرائضي ،',
 '، أب',
 'أب بكر

In [86]:
find_ngram_in_text(clean_example1,'من ناحية')

[True]

In [98]:
yaqutz_pd

,#,PlaceName,Description,Latitude,Longitude,Type1,Type2,TypeFull,PlaceType,TypeEng,...,wikilat,wikilong,TurLat,TurLong,TurNameTr,TurNameEng,TurNameAr,TurType,TurRegionEng,TurRegionTr
0,0,آبار الأعراب,# جمع بئر. يقال في جمعها آبار وبئار وأبار: موض...,NaN,NaN,موضع,موضع,موضع,موضع,place,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,آبج,# بفتح الهمزة وبعد الألف باء موحدة مفتوحة وجيم...,NaN,NaN,موضع,موضع,موضع,موضع,place,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,آبر,# بفتح الهمزة وسكون الألف وضم الباء الموحدة ور...,NaN,NaN,قرية,قرية,قرية,قرية,village,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,آبسكون,# بفتح الهمزة وسكون الألف وفتح الباء الموحدة و...,37.027720,54.043270,بليدة,بليدة,بليدة,بليدة,small town (f),...,NaN,NaN,37.02772,54.04327,Ābaskūn,Abaskun,آبسكون,towns,Daylam,al-Daylam
4,4,آبل,# بفتح الهمزة وبعد الألف باء مكسورة ولام:\n<br...,34.637222,36.676111,أربعة مواضع وفي الحديث,أربعة مواضع وفي الحديث,أربعة مواضع وفي الحديث,أربعة مواضع,NaN,...,34.637222,36.676111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12449,12449,يونان,# بالضم ثم السكون، ونونين بينهما ألف:\n<br />#...,39.000000,22.000000,موضع,موضع,موضع,موضع,place,...,39.000000,22.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12450,12450,يؤيؤ,# بالضم ثم السكون ثم مثله، يوم يؤيؤ: وهو يوم ا...,NaN,NaN,وهو يوم الأواق من,وهو يوم الأواق من,يوم الأواق من,يوم,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12451,12451,يهرع,# بالفتح، قوله تعالى: وجاءه قومه يهرعون إليه، ...,NaN,NaN,وجاءه,وجاءه,وجاءه,وجاءه,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12452,12452,يهودية,# نسبة إلى اليهود في موضعين: أحدهما محلة بجرجا...,35.934700,64.838570,أحدهما,أحدهما,أحدهما,أحدهما,NaN,...,32.500000,34.900000,35.93470,64.83857,"al-Yahūdiyyaŧ, Ǧakūḏān","al-Yahudiyya, Jakudhan",اليهودية,waystations,Khurasan,Ḫurāsān


In [100]:
df10 =yaqutz_pd[['PlaceName', 'Description']].copy()

In [101]:
df10['steam_Description'] = df10['Description'].apply(lambda x:clean_text(x))

In [103]:
df10['clean_Description'] = df10['Description'].apply(lambda x:clean_text(x,stem=False))

In [133]:
def blahblah(text):
    tokens = text.split(' ')
    werd=[]
    for ix,token in zip(range(len(tokens)),tokens):
        if(token in pd_types):
            if(ix < len(tokens) -1 ):
                werd.append(tokens[ix+1])
    return werd

In [116]:
df10['steam_Description'].head(50).apply(lambda x : find_mn_patter(x) )

0       []
1       []
2     [10]
3       []
4       []
5       []
6     [14]
7     [28]
8       []
9       []
10      []
11      []
12    [10]
13      []
14      []
15     [6]
16    [13]
17      []
18      []
19      []
20      []
21      []
22    [33]
23      []
24      []
25      []
26      []
27      []
28      []
29      []
30      []
31      []
32      []
33      []
34      []
35      []
36      []
37      []
38      []
39      []
40      []
41      []
42      []
43      []
44      []
45      []
46    [33]
47      []
48      []
49      []
Name: steam_Description, dtype: object

In [134]:
df10['followupword'] = df10['steam_Description'].apply(lambda x : blahblah(x))
df10['followupword']

0                            [., بئار, بين, أيض, من, واسط]
1                                               [في, معجم]
2                                             [من, سجستان]
3                      [:, على, طبرستان, آبسكون, شام, دار]
4                                                      [.]
                               ...                        
12449                                          [من, بعلبك]
12450                                                   []
12451                                                  [.]
12452    [:, جرجان, أخرج, كان, لا, إلا, تراب, زال, من, ...
12453                     [مهمل, رقيق, يمن, كان, كان, كان]
Name: followupword, Length: 12454, dtype: object

In [136]:
a = pd.Series([item for sublist in df10.followupword for item in sublist])

In [137]:
df22 = a.value_counts().sort_index().rename_axis('followupword').reset_index(name='f')

In [146]:
df22.sort_values(by=['f'],ascending=False).to_csv('word_after_type_count.csv')

Index(['followupword', 'f'], dtype='object')

In [145]:
df22.to_csv('word_after_type_count.csv')